<a href="https://colab.research.google.com/github/KhizarAziz/C3AE_age_estimation_notebook/blob/master/C3AE_Notebook_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> ## **`Clone Repo`**



In [ ]:
!git clone https://github.com/KhizarAziz/C3AE_age_estimation_notebook.git



> ## **`Download Datasets`**



In [ ]:
#download imdb
# !wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
#download WIKI
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

In [ ]:
#extract IMDB-WIKI datasets
# !tar -xvf /content/imdb_crop.tar -C /content/C3AE/dataset
!tar -xvf /content/wiki_crop.tar -C /content/C3AE_keras/datasets/
# morph data
# !tar -xvf Morph.tar -C /content/C3AE/dataset



> ## **`Preprocess Datasets`**



In [ ]:
cd /content/C3AE_keras

In [ ]:
#Wiki-IDMB
!python /content/C3AE_keras/preprocessing_scripts/preprocess_WIKI-IMDB.py
#Morph
# !python /content/C3AE_keras/preprocessing_scripts/preprocess_Morph.py



> ## **`Train`**



In [ ]:
cd /content/C3AE_keras

In [ ]:
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import dlib
from net_training import C3AE_net,training_utils
import feather
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras.losses import kl_divergence,mae
from keras.metrics import mae
import tensorflow as tf

In [ ]:
# initializing params
category = 10
dropout = 0.2
seed = 2019
category = category + 2
interval = 10

In [ ]:
# Loading dataset (from .feather file)
dataset_dir = Path('/content/C3AE_keras/datasets/wiki_crop/')
dataset_df = pd.DataFrame(columns=["age", "gender", "image", "org_box", "trible_box", "landmarks", "roll", "yaw", "pitch"])
for fnames in dataset_dir.glob('*.feather'):
  df_chunk = feather.read_dataframe(dataset_dir.joinpath(fnames))
  dataset_df = pd.concat([dataset_df,df_chunk],ignore_index=True)
dataset_df.shape

In [ ]:
#validation split using sklearn.model_selection.train_test_split
trainset, testset = train_test_split(dataset_df, train_size=0.8, test_size=0.2, random_state=seed)

In [ ]:
# making a generator for image and dataset. inside generator we 
#load img, crop faces (3 sizes -> big,middle,small) then tranform (if aumentation is true) rotate,birhgt etc ect
# convert age into 2 point represenation (like on hot encoding)
input_imgs_shape = (64,64) # input image shape to network
batch_size = 32 
require_data_augmentation = False # if this is true, then random noise will be added to images, e.g rotation, brightness, hue etc
train_gen = C3AE_net.preprocessing(trainset,batch_size=batch_size, category=category, interval=interval,input_imgs_shape=input_imgs_shape,augmentation=require_data_augmentation,dropout=dropout)
validation_gen = C3AE_net.preprocessing(testset, augmentation=require_data_augmentation, category=category, interval=interval)
# print(trainset.groupby(["age"])["age"].agg("count"))

In [ ]:
#checking generator working fine!
index = 10
for i in train_gen:
  print(i[1][0][index],i[1][1][index])
  break

29.0 [0.  0.  0.1 0.9 0.  0.  0.  0.  0.  0.  0.  0. ]


In [ ]:
# just to check how versatile is our dataset
# get distribution of ages i.e how many number of rows for each interval i.e lets say for 0-10 years age we have 5000 entries(rows)
age_distribution = [trainset["age"][(trainset.age >= x -10) & (trainset.age <= x)].count() for x in range(10, 101, 10)]
age_distribution = [age_distribution[0]] + age_distribution + [age_distribution[-1]]
print(age_distribution)

In [ ]:
se_net = True # if true, then network will contain SE_Block (GAP + FC + mul), which improved results.
using_white_norm = True # if true, then network will contain white_normlization block, which will normalize image brightness and colors. (for better accuracy)
models = C3AE_net.build_net(Categories = category, using_SE=se_net, using_white_norm=using_white_norm)

# add pretrain weights if exist
pretrain_weights_path = Path("/content/C3AE_keras/models_saved/wiki_trained__age_mae_7.7194.h5")
if Path.is_file(pretrain_weights_path):
  models.load_weights(pretrain_weights_path)

In [ ]:
lr = 0.1
adam = Adam(lr=lr)
#cate_weight = K.variable(params.weight_factor)
weight_factor = 10
models.compile(
    optimizer=adam,
    loss = {'W1':kl_divergence,'age':mae},
    metrics={"age": mae},
    loss_weights={'W1':weight_factor, 'age': 1}
)

In [ ]:
class lr_Callback(tf.keras.callbacks.Callback):
  def on_epoch_begin(self, batch, logs={}):
      # Get the current learning rate from model's optimizer.
      lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
      print('current epoch lr is: {} '.format(lr))

In [ ]:
save_path = '/content/C3AE_keras/models_saved/'
callbacks = [
  ModelCheckpoint(save_path+'WIKI-weights.{epoch:02d}-{val_age_mean_absolute_error:.2f}.hdf5',
                  monitor='val_age_mean_absolute_error',
                  verbose = 1,
                  save_best_only=True,
                  model ='min'),
             
  ReduceLROnPlateau(monitor='val_age_mean_absolute_error', #considered metric
                    factor = 0.5, # learning_rate * factor (multiply lr 0.5 to reduce it by 50%)
                    patience = 2,# if loss didnt improve this much epochs, then update lr
                    min_delta = 0.01, # if model didnt improve this much
                    cooldown = 1, # after lr updated, wait this many epochs before apply new lr
                    min_lr = 0.001, # lower limit of lr
                    mode = 'min'), # loss should go up (max) or down (min)
  lr_Callback() # print lr on each epoch, to analyze lr performance.
]

In [ ]:
epochs=2
history = models.fit(train_gen, steps_per_epoch=len(trainset) / batch_size, epochs=epochs, callbacks=callbacks, validation_data=validation_gen, validation_steps=len(testset) / batch_size * 3)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['age_loss'])
plt.plot(history.history['val_age_loss'])
plt.legend(['age_loss', 'val_age_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['W1_loss'])
plt.plot(history.history['val_W1_loss'])
plt.legend(['W1_loss', 'val_W1_loss'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['age_mean_absolute_error'])
plt.plot(history.history['val_age_mean_absolute_error'])
plt.legend(['age_mean_absolute_error', 'val_age_mean_absolute_error'], loc='upper left')
plt.show()



> ## **`Inference`**






In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/C3AE_keras/detector/shape_predictor_68_face_landmarks.dat")
def gen_face(image):
  face_rect_list = detector(image)
  xmin, ymin, xmax, ymax = face_rect_list[0].left() , face_rect_list[0].top(), face_rect_list[0].right(), face_rect_list[0].bottom() # face_rect is dlib.rectangle object, so extracting values from it
  lmarks_list = dlib.full_object_detections()
  for face_rect in face_rect_list:
    lmarks_list.append(predictor(image, face_rect)) # getting landmarks as a list of objects
  return np.array([xmin, ymin, xmax, ymax]), lmarks_list

def gen_boundbox(box, landmark):
    # getting 3 boxes for face, as required in paper... i.e feed 3 different sized images to network (R,G,B) 
    xmin, ymin, xmax, ymax = box # box is [xmin, ymin, xmax, ymax]
    w, h = xmax - xmin, ymax - ymin
    nose_x, nose_y = (landmark.parts()[30].x, landmark.parts()[30].y) # calculating nose center point, so the triple boxes will be cropped according to nose point
    w_h_margin = abs(w - h)
    top2nose = nose_y - ymin
    # Contains the smallest frame
    return np.array([
        [(xmin - w_h_margin, ymin - w_h_margin), (xmax + w_h_margin, ymax + w_h_margin)],  # out
        [(nose_x - top2nose, nose_y - top2nose), (nose_x + top2nose, nose_y + top2nose)],  # middle
        [(nose_x - w//2, nose_y - w//2), (nose_x + w//2, nose_y + w//2)]  # inner box
    ])

In [ ]:
img = cv2.imread('/content/test.jpg',)
model = models

In [ ]:
try:
    bounds, lmarks = gen_face(img)
except Exception as e:
  print(e)

padding = 200
new_bd_img = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_CONSTANT)

In [ ]:
for pidx,landmarks in enumerate(lmarks):
    trible_box = gen_boundbox(bounds, landmarks)
    tri_imgs = []
    for bbox in trible_box:
        bbox = bbox 
        h_min, w_min = bbox[0]
        h_max, w_max = bbox[1]
        cropped_resized_img = cv2.resize(new_bd_img[w_min:w_max, h_min:h_max], (64, 64))
        final_img = np.expand_dims(cropped_resized_img,axis=0)
        tri_imgs.append(final_img)
    print(np.array(tri_imgs).shape)

In [ ]:
result = models.predict(tri_imgs)

In [ ]:
print("Age is: ",result[0][0][0])

Age is:  26.070438
